In [ ]:
import pandas as pd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import time
import obspy
import numpy as np

# 0. 버전 확인 (1.4.2가 나와야 함)
print(f"--- 현재 obspy 버전: {obspy.__version__} ---")

client = Client("IRIS")

# --- 1. 2024년도 지진 '검색' ---
# (ID를 모르므로, 2024년 1월 1일 일본 노토 반도 지진(M7.5)을 검색)
print("--- 1. 2024년 1월 1일 (M3.0+) 지진 검색 중... ---")
search_start = UTCDateTime("2024-01-01T00:00:00")
search_end = UTCDateTime("2024-01-02T00:00:00") # 1월 1일 하루 동안

origin_time = None
event_lat = None
event_lon = None
event_id_str = "N/A"

try:
    cat = client.get_events(
        starttime=search_start,
        endtime=search_end,
        minmagnitude=3.0 # M7.0 이상만 검색
    )

    if not cat:
        print("  > 2024년 1월 1일에 M3.0+ 지진을 찾을 수 없습니다.")
        print("  > 다른 날짜로 시도해보세요.")
        exit()

    # 검색된 첫 번째 이벤트(가장 큰 이벤트)를 사용
    event = cat[0]

    # 파형 수집에 필요한 정보 추출
    origin_time = event.origins[0].time
    event_lat = event.origins[0].latitude
    event_lon = event.origins[0].longitude
    event_mag = event.magnitudes[0].mag
    # event.resource_id는 객체일 수 있으므로 .id 속성 확인
    event_id_str = event.resource_id.id if event.resource_id else "ID_NOT_FOUND"

    print(f"  > 검색된 이벤트: {event.event_descriptions[0].text}")
    print(f"  > Event ID: {event_id_str}")
    print(f"  > 발생 시각 (UTC): {origin_time}")
    print(f"  > 규모: {event_mag}")
    print(f"  > 위치: {event_lat}, {event_lon}")

except Exception as e:
    print(f"  > get_events() 오류 (이벤트 검색 실패): {e}")
    exit()

# --- 2. '파형(Waveform)' 데이터 수집 ---
# (1단계에서 찾은 시간, 위치를 기준으로 파형을 요청)

print(f"\n--- 2. '{event_id_str}'의 파형(Waveform) 데이터 수집 시도... ---")
print("(조건: 발생 후 10분간, 진앙 5도 이내, BH? 채널)")

st = None # 파형 묶음 (Stream)

try:
    st = client.get_waveforms(
        network="*",        # 모든 네트워크
        station="*",      # 모든 관측소
        location="*",     # 모든 위치 코드
        channel="BH?",      # 광대역 채널 (BHZ, BHN, BHE)
        starttime=origin_time,              # 1. 검색된 지진 발생 시각
        endtime=origin_time + 600,          # 10분(600초) 동안
        latitude=event_lat,                 # 2. 검색된 위도
        longitude=event_lon,                # 3. 검색된 경도
        maxradius=5.0                       # 반경 5도 이내
    )

    if st:
        print(f"\n--- 3. 파형 수집 성공! ---")
        print(f"  > 총 {len(st)} 개의 파형(채널)을 다운로드했습니다.")

        # --- 4. "수치화된 데이터" (Number Array) 확인 ---
        print("\n--- 4. '수치화된 데이터' (숫자 배열) 확인 ---")

        tr = st[0] # 첫 번째 파형을 샘플로 확인
        print(f"  > 첫 번째 파형 ID: {tr.id}")

        # <<< 이것이 바로 모델에 넣을 "숫자 배열"입니다 >>>
        number_array = tr.data

        print(f"  > 이 파형의 총 숫자 개수: {len(number_array)} 개")
        print(f"  > 실제 숫자 배열 (앞 10개): {number_array[:10]}")

        # --- 5. 특징(Feature) 계산 예시 ---
        print("\n--- 5. 이 '숫자 배열'로 특징(Feature) 계산하기 (예시) ---")

        max_amplitude = np.max(np.abs(number_array))

        print(f"  > (예시 특징) 최대 진폭 (Max Amplitude): {max_amplitude:.2f}")

    else:
        print(f"\n--- 3. 수집 실패 (데이터 없음) ---")
        print("  > 이 조건(2024년 1월 1일)에 맞는 파형 데이터가 서버에 없습니다.")

except AttributeError as ae:
    print(f"\n--- !!! 심각한 오류 발생 (AttributeError) !!! ---")
    print(f"  > 오류: {ae}")
    print(f"  > 원인: 'obspy.Client'가 'get_waveforms' 함수도 찾지 못합니다.")
    print(f"  > 해결: Python 3.11 가상환경 폴더(venv)를 삭제하고 2단계부터 다시 시도해야 합니다.")

except Exception as e:
    print(f"\n--- !!! 오류 발생 !!! ---")
    print(f"  > 오류: {e}")
    if "No data available" in str(e) or "204" in str(e):
        print("  > (원인: 이 조건에 맞는 데이터가 서버에 없습니다.)")
    else:
        print("  > (원인: 그 외 오류. 네트워크 문제일 수 있습니다.)")

In [ ]:
import pandas as pd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import time
import obspy
import numpy as np
import os

print(f"--- 현재 obspy 버전: {obspy.__version__} ---")
client = Client("IRIS")

# --- 1. 2024년도 지진 '검색' ---
print("--- 1. 2024년 1월 1일 (M3.0+) 지진 검색 중... ---")
search_start = UTCDateTime("2024-02-01T00:00:00")
search_end = UTCDateTime("2024-03-01T00:00:00")
min_mag = 3.0

results_list = []

try:
    cat = client.get_events(
        starttime=search_start,
        endtime=search_end,
        minmagnitude=min_mag
    )
    if not cat:
        print(f"  > {search_start.date}에 M{min_mag}+ 지진을 찾을 수 없습니다.")
        exit()

    print(f"  > 총 {len(cat)} 개의 M{min_mag}+ 지진을 찾았습니다. 파형 수집을 시작합니다.")

    # --- 2. '모든' 지진을 순회 ---
    for i, event in enumerate(cat):

        event_time = None
        event_lat = None
        event_lon = None
        event_mag = None
        event_depth_km = None
        event_id_str = "N/A"

        # 검색할 반경 (단위: 도)
        search_radius_deg = 2.0

        try:
            # --- 3. 기본 지진 정보 추출 ---
            origin = event.origins[0]
            event_time = origin.time
            event_lat = origin.latitude
            event_lon = origin.longitude
            event_depth_km = origin.depth / 1000.0 if origin.depth else 0.0
            event_mag = event.magnitudes[0].mag if event.magnitudes else 0.0

            if event.resource_id:
                event_id_str = str(event.resource_id.id).split('=')[-1]
            else:
                event_id_str = f"event_{i}"

            print(f"\n--- [{i+1}/{len(cat)}] 이벤트 처리 중 (ID: {event_id_str}, Mag: {event_mag}) ---")

            # --- 4. (!!! 여기가 수정된 부분 1 !!!) ---
            # 먼저 지진 근처의 '관측소(Station)' 목록을 가져옵니다.
            print(f"  > 4a. 반경 {search_radius_deg}도 이내의 관측소 검색 중...")

            stations_inventory = client.get_stations(
                starttime=event_time,
                endtime=event_time + 300, # 5분
                latitude=event_lat,       # get_stations는 이 파라미터가 작동해야 합니다.
                longitude=event_lon,
                maxradius=search_radius_deg,
                level="station"
            )

            if not stations_inventory or len(stations_inventory) == 0:
                print("  > 근처에 관측소를 찾지 못했습니다. (데이터 없음). 다음 이벤트로 넘어갑니다.")
                continue

            # 관측소 목록에서 '네트워크'와 '관측소' 코드만 추출합니다.
            network_codes = set()
            station_codes = set()
            for network in stations_inventory:
                network_codes.add(network.code)
                for station in network:
                    station_codes.add(station.code)

            if not station_codes:
                 print("  > 관측소 코드를 추출하지 못했습니다. 다음 이벤트로 넘어갑니다.")
                 continue

            # 콤마(,)로 구분된 문자열로 변환 (e.g., "JP,KR", "HON,INU")
            net_str = ",".join(network_codes)
            sta_str = ",".join(station_codes)

            print(f"  > 4b. 찾은 관측소({len(station_codes)}개)의 파형 요청 중...")

            # --- 5. (!!! 여기가 수정된 부분 2 !!!) ---
            # 'minlatitude' 대신 'network', 'station' 코드로 파형 요청
            st = client.get_waveforms(
                network=net_str,    # (수정) 네트워크 코드로 필터
                station=sta_str,    # (수정) 관측소 코드로 필터
                location="*",
                channel="BHZ",
                starttime=event_time,
                endtime=event_time + 300
                # (삭제) minlatitude, maxlatitude, minlongitude, maxlongitude
            )

            max_amplitude = 0.0

            if st:
                max_amps_per_station = [np.max(np.abs(tr.data)) for tr in st if len(tr.data) > 0]
                if max_amps_per_station:
                    max_amplitude = np.max(max_amps_per_station)
                print(f"  > 파형 수집 성공. 최대 진폭 (Max Amp): {max_amplitude:.2f}")
            else:
                print(f"  > 이 이벤트의 파형 데이터를 찾을 수 없습니다 (No data available).")

            # --- 6. CSV 저장을 위해 리스트에 '결과' 추가 ---
            results_list.append({
                "event_id": event_id_str,
                "time_utc": event_time.isoformat(),
                "latitude": event_lat,
                "longitude": event_lon,
                "magnitude": event_mag,
                "depth_km": event_depth_km,
                "max_amplitude_bhz": max_amplitude
            })

        except Exception as e:
            # 만약 get_stations()에서도 'latitude' 오류가 난다면, obspy.Client 자체가 완전히 고장 난 것입니다.
            print(f"  > 이벤트 처리 중 치명적 오류 발생: {e}")
            continue

except Exception as e:
    print(f"  > get_events() 오류 (이벤트 검색 실패): {e}")
    exit()

# --- 7. 모든 작업 완료 후, 리스트를 Pandas DataFrame으로 변환 ---
print("\n--- 7. 모든 이벤트 처리 완료. CSV 파일로 저장합니다. ---")

if results_list:
    df = pd.DataFrame(results_list)
    output_filename = f"earthquake_features_{search_start.date}.csv"
    output_path = os.path.join(os.getcwd(), output_filename)
    df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"  > 성공! 총 {len(df)} 개의 이벤트 데이터를")
    print(f"  > '{output_path}' 파일로 저장했습니다.")
    print("\n[CSV 샘플 데이터]")
    print(df.head())
else:
    print("  > 저장할 데이터가 없습니다. (모든 이벤트에서 파형 수집 실패)")

print("\n--- 작업 종료 ---")

In [ ]:
import pandas as pd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import time
import obspy
import numpy as np
import os

print(f"--- 현재 obspy 버전: {obspy.__version__} ---")
client = Client("IRIS")

# --- 1. 2024년도 지진 '검색' ---
print("--- 1. 2024년 1월 1일 (M3.0+) 지진 검색 중... ---")
search_start = UTCDateTime("2024-01-01T00:00:00")
search_end = UTCDateTime("2024-01-02T00:00:00")
min_mag = 3.0

results_list = []

try:
    # (수정) 쓰나미 정보를 포함할 수 있도록 includeall=True 추가
    cat = client.get_events(
        starttime=search_start,
        endtime=search_end,
        minmagnitude=min_mag,
        includeallorigins=True # 혹시 모를 상세 정보를 위해
    )
    if not cat:
        print(f"  > {search_start.date}에 M{min_mag}+ 지진을 찾을 수 없습니다.")
        exit()

    print(f"  > 총 {len(cat)} 개의 M{min_mag}+ 지진을 찾았습니다. 파형 수집을 시작합니다.")

    # --- 2. '모든' 지진을 순회 ---
    for i, event in enumerate(cat):

        event_time = None
        event_lat = None
        event_lon = None
        event_mag = None
        event_depth_km = None
        event_id_str = "N/A"

        search_radius_deg = 2.0

        try:
            # --- 3. 기본 지진 정보 추출 ---
            origin = event.origins[0]
            event_time = origin.time
            event_lat = origin.latitude
            event_lon = origin.longitude
            event_depth_km = origin.depth / 1000.0 if origin.depth else 0.0
            event_mag = event.magnitudes[0].mag if event.magnitudes else 0.0

            if event.resource_id:
                event_id_str = str(event.resource_id.id).split('=')[-1]
            else:
                event_id_str = f"event_{i}"

            print(f"\n--- [{i+1}/{len(cat)}] 이벤트 처리 중 (ID: {event_id_str}, Mag: {event_mag}) ---")

            # --- (!!! 여기가 새로 추가된 부분 !!!) ---
            # --- 4. 쓰나미 정보 확인 ---
            tsunami_flag = 0 # 0 = 쓰나미 아님, 1 = 쓰나미 가능성

            # 4a. 이벤트 타입 확인
            if event.event_type == "tsunami":
                tsunami_flag = 1

            # 4b. 이벤트 설명(description) 텍스트 확인
            if tsunami_flag == 0 and event.event_descriptions:
                for desc in event.event_descriptions:
                    if "tsunami" in desc.text.lower():
                        tsunami_flag = 1
                        break # 'tsunami' 단어를 찾았으면 중단

            if tsunami_flag == 1:
                print("  > !!! 쓰나미(tsunami) 키워드 발견! !!!")
            # --- (추가 끝) ---


            # --- 5. 근처 '관측소(Station)' 목록 검색 ---
            print(f"  > 5a. 반경 {search_radius_deg}도 이내의 관측소 검색 중...")

            stations_inventory = client.get_stations(
                starttime=event_time,
                endtime=event_time + 300,
                latitude=event_lat,
                longitude=event_lon,
                maxradius=search_radius_deg,
                level="station"
            )

            if not stations_inventory or len(stations_inventory) == 0:
                print("  > 근처에 관측소를 찾지 못했습니다. (데이터 없음).")
                # (수정) 관측소가 없어도 지진 정보(쓰나미 플래그 포함)는 저장
                max_amplitude = 0.0 # 파형이 없으므로 0

            else:
                # --- 6. 파형(Waveform) 데이터 수집 ---
                network_codes = {net.code for net in stations_inventory}
                station_codes = {sta.code for net in stations_inventory for sta in net}

                net_str = ",".join(network_codes)
                sta_str = ",".join(station_codes)

                print(f"  > 6b. 찾은 관측소({len(station_codes)}개)의 파형 요청 중...")

                st = client.get_waveforms(
                    network=net_str,
                    station=sta_str,
                    location="*",
                    channel="BHZ",
                    starttime=event_time,
                    endtime=event_time + 300
                )

                max_amplitude = 0.0

                if st:
                    max_amps_per_station = [np.max(np.abs(tr.data)) for tr in st if len(tr.data) > 0]
                    if max_amps_per_station:
                        max_amplitude = np.max(max_amps_per_station)
                    print(f"  > 파형 수집 성공. 최대 진폭 (Max Amp): {max_amplitude:.2f}")
                else:
                    print(f"  > 이 이벤트의 파형 데이터를 찾을 수 없습니다 (No data available).")

            # --- 7. CSV 저장을 위해 리스트에 '결과' 추가 ---
            results_list.append({
                "event_id": event_id_str,
                "time_utc": event_time.isoformat(),
                "latitude": event_lat,
                "longitude": event_lon,
                "magnitude": event_mag,
                "depth_km": event_depth_km,
                "max_amplitude_bhz": max_amplitude,
                "tsunami_flag": tsunami_flag  # (!!! 추가된 컬럼 !!!)
            })

        except Exception as e:
            print(f"  > 이벤트 처리 중 치명적 오류 발생: {e}")
            continue

except Exception as e:
    print(f"  > get_events() 오류 (이벤트 검색 실패): {e}")
    exit()

# --- 8. 모든 작업 완료 후, 리스트를 Pandas DataFrame으로 변환 ---
print("\n--- 8. 모든 이벤트 처리 완료. CSV 파일로 저장합니다. ---")

if results_list:
    df = pd.DataFrame(results_list)
    output_filename = f"earthquake_features_tsunami_{search_start.date}.csv" # 파일 이름 변경
    output_path = os.path.join(os.getcwd(), output_filename)
    df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"  > 성공! 총 {len(df)} 개의 이벤트 데이터를")
    print(f"  > '{output_path}' 파일로 저장했습니다.")
    print("\n[CSV 샘플 데이터]")
    print(df.head())
else:
    print("  > 저장할 데이터가 없습니다. (모든 이벤트에서 파형 수집 실패)")

print("\n--- 작업 종료 ---")

In [ ]:
import requests
import json
from obspy import UTCDateTime

# --- 1. CSV의 첫 번째 줄 데이터 (규모 4.3) ---
event_lat_from_iris = 37.3362
event_lon_from_iris = 136.9617
event_time_from_iris = UTCDateTime("2024-01-01T22:13:30.239000")

print(f"--- IRIS에서 찾은 이벤트 정보 (M 4.3) ---")
print(f"Time: {event_time_from_iris}")
print(f"Lat: {event_lat_from_iris}, Lon: {event_lon_from_iris}\n")

# --- 2. 이 이벤트를 위한 '시공간 창' 정의 ---
search_start_time = event_time_from_iris - 60  # 1분 전
search_end_time = event_time_from_iris + 60    # 1분 후
search_radius_km = 100 # 100km 반경 (기관 간 위치 오차 감안)

# --- 3. USGS API로 '좁은 검색' 요청 ---
url = (
    f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
    f"&starttime={search_start_time.isoformat()}"
    f"&endtime={search_end_time.isoformat()}"
    f"&latitude={event_lat_from_iris}"
    f"&longitude={event_lon_from_iris}"
    f"&maxradiuskm={search_radius_km}"
    # (!!!) 바로 이 부분! 최소 규모 필터를 제거해야 합니다 (!!!)
    # f"&minmagnitude=4.5"  <- 이 코드가 문제였습니다.
)

print(f"--- USGS에 '좁은 검색' 요청 ---")
print(f"URL: {url}\n")

try:
    response = requests.get(url)
    data = response.json()

    num_found = len(data['features'])

    if num_found == 1:
        # --- 4. 정확히 1개의 이벤트를 찾았을 때 (성공!) ---
        feature = data['features'][0]
        props = feature['properties']

        usgs_tsunami_flag = props['tsunami']
        usgs_event_id = feature['id']
        usgs_mag = props['mag']

        print(f"  > 성공! USGS에서 매칭되는 이벤트 1개를 찾았습니다.")
        print(f"  > USGS ID: {usgs_event_id}") # 아마 'us7000lsyp'가 나올 것입니다.
        print(f"  > USGS Mag: {usgs_mag}")
        print(f"  > *** 공식 쓰나미 플래그 (tsunami): {usgs_tsunami_flag} ***")

        if usgs_tsunami_flag == 1:
            print("  > (결과: 쓰나미 발생 이벤트)")
        else:
            print("  > (결과: 쓰나미 미발생 이벤트)")

    elif num_found > 1:
        print(f"  > 오류: 좁은 범위에서 {num_found}개의 이벤트가 발견되었습니다.")
        # 이 경우, magnitude가 가장 비슷한 것을 고르는 로직이 추가로 필요합니다.
    else:
        print("  > 오류: USGS 카탈로그에서 매칭되는 이벤트를 찾지 못했습니다.")

except Exception as e:
    print(f"  > 요청 중 오류 발생: {e}")

In [ ]:
import pandas as pd
import requests
import time
import numpy as np
import os
from datetime import datetime, timedelta # obspy 대신 표준 라이브러리 사용

# --- 1. 파일 불러오기 ---
input_filename = "earthquake_features_2024-01-01.csv"
try:
    df = pd.read_csv(input_filename)
    print(f"--- '{input_filename}' 파일을 성공적으로 불러왔습니다. (총 {len(df)}개 이벤트) ---")
except FileNotFoundError:
    print(f"--- 오류: '{input_filename}' 파일을 찾을 수 없습니다. ---")
    exit() # 스크립트 종료

# --- 2. USGS API 요청을 위한 설정 ---
usgs_api_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
search_radius_km = 100  # IRIS와 USGS 간의 위치 오차를 고려한 검색 반경
time_window_seconds = 60 # IRIS와 USGS 간의 시간 오차를 고려한 검색 시간 (±60초)

results_tsunami_flags = [] # USGS 플래그를 저장할 리스트

print(f"--- USGS API 조회를 시작합니다 (총 {len(df)}개 이벤트). 이벤트당 약 1초가 소요됩니다. ---")

# --- 3. DataFrame의 '모든 행'을 순회 ---
for index, row in df.iterrows():

    # CSV에서 현재 이벤트 정보 추출
    event_lat = row['latitude']
    event_lon = row['longitude']
    event_mag = row['magnitude']
    event_time_str = row['time_utc'] # 문자열로 시간 가져오기

    try:
        # CSV의 시간 형식: '2024-01-01T22:27:01.373000'
        event_time_dt = datetime.strptime(event_time_str, '%Y-%m-%dT%H:%M:%S.%f')
    except ValueError:
        # 가끔 마이크로초(.f)가 없는 시간 형식을 대비
        try:
            event_time_dt = datetime.strptime(event_time_str, '%Y-%m-%dT%H:%M:%S')
        except Exception as e:
            print(f"  > 시간 파싱 오류 (행 {index}): {e}. 이 행은 건너뜁니다.")
            results_tsunami_flags.append(np.nan)
            continue

    print(f"\n--- [ {index + 1} / {len(df)} ] 처리 중 (IRIS Mag: {event_mag}) ---")
    print(f"  > IRIS 시간: {event_time_dt.isoformat()}")

    # USGS 검색을 위한 '시공간 창' 설정 (timedelta 사용)
    search_start_dt = event_time_dt - timedelta(seconds=time_window_seconds)
    search_end_dt = event_time_dt + timedelta(seconds=time_window_seconds)

    # API가 요구하는 ISO 형식으로 변환 (UTC 표기 'Z' 추가)
    search_start_iso = search_start_dt.isoformat() + "Z"
    search_end_iso = search_end_dt.isoformat() + "Z"

    # API 요청 파라미터 (!!! minmagnitude 필터 제거 !!!)
    params = {
        'format': 'geojson',
        'starttime': search_start_iso,
        'endtime': search_end_iso,
        'latitude': event_lat,
        'longitude': event_lon,
        'maxradiuskm': search_radius_km
    }

    try:
        # --- 4. USGS API 호출 ---
        response = requests.get(usgs_api_url, params=params, timeout=10)

        usgs_tsunami_flag = np.nan # 기본값은 '데이터 없음'

        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])

            # --- 5. API 응답 결과 처리 ---
            if len(features) == 0:
                # Case 1: 좁은 범위에서 매칭되는 이벤트 '없음'
                print("  > USGS 매칭 실패 (0개). 쓰나미 플래그 '0'으로 설정.")
                usgs_tsunami_flag = 0 # 매칭이 안되면 쓰나미가 아니라고 가정

            elif len(features) == 1:
                # Case 2: 1개로 '정확히' 매칭됨 (Best case)
                props = features[0]['properties']
                usgs_tsunami_flag = props['tsunami']
                print(f"  > USGS 매칭 성공 (1개)! (ID: {features[0]['id']}, Mag: {props['mag']})")
                print(f"  > *** 공식 쓰나미 플래그: {usgs_tsunami_flag} ***")

            else:
                # Case 3: 2개 이상 '모호하게' 매칭됨
                print(f"  > USGS 매칭 모호함 ({len(features)}개 발견). 규모가 가장 비슷한 이벤트로 매칭 시도...")

                best_match = None
                min_mag_diff = float('inf')

                # IRIS의 규모(event_mag)와 가장 차이가 적게 나는 이벤트를 찾는다
                for f in features:
                    usgs_mag = f['properties']['mag']
                    if usgs_mag is None: continue

                    mag_diff = abs(usgs_mag - event_mag)

                    if mag_diff < min_mag_diff:
                        min_mag_diff = mag_diff
                        best_match = f

                if best_match:
                    props = best_match['properties']
                    usgs_tsunami_flag = props['tsunami']
                    print(f"  > > 최적 매칭: (ID: {best_match['id']}, Mag: {props['mag']})")
                    print(f"  > *** 공식 쓰나미 플래그: {usgs_tsunami_flag} ***")
                else:
                    print("  > > 최적 매칭 실패. 플래그 '0'으로 설정.")
                    usgs_tsunami_flag = 0 # 그래도 못찾으면 0

        else:
            # API 자체가 실패한 경우 (e.g., 500 서버 오류)
            print(f"  > USGS API 요청 실패 (HTTP Status: {response.status_code}).")
            usgs_tsunami_flag = np.nan # '0'이 아니라 '데이터 없음'으로 표기

    except requests.RequestException as e:
        # 네트워크 연결 오류 등
        print(f"  > API 요청 중 예외 발생: {e}")
        usgs_tsunami_flag = np.nan # '데이터 없음'

    # 최종 플래그를 리스트에 추가
    results_tsunami_flags.append(usgs_tsunami_flag)

    # !!! USGS API 서버 과부하를 막기 위해 1초 대기 (매우 중요) !!!
    time.sleep(1)

# --- 6. 원본 DataFrame에 새로운 컬럼 추가 ---
df['tsunami_flag_usgs'] = results_tsunami_flags

# --- 7. 새 파일로 저장 ---
output_filename = f"earthquake_features_with_tsunami_usgs.csv"
output_path = os.path.join(os.getcwd(), output_filename)
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n--- 작업 완료 ---")
print(f"  > '{output_path}' 파일에 USGS 쓰나미 플래그가 추가된 데이터를 저장했습니다.")
print("\n[최종 데이터 샘플 (tsunami_flag_usgs 컬럼 확인)]")
print(df.head())

# 쓰나미 플래그가 1인 (즉, 쓰나미가 발생한) 이벤트가 있었는지 확인
tsunami_events = df[df['tsunami_flag_usgs'] == 1]
if not tsunami_events.empty:
    print("\n[!!! 쓰나미(1)로 확인된 이벤트 !!!]")
    print(tsunami_events[['time_utc', 'magnitude', 'tsunami_flag_usgs']])
else:
    print("\n[확인: 이 데이터셋에서 쓰나미(1)로 확인된 이벤트는 없습니다.]")

In [ ]:
import pandas as pd
import requests
import time
import numpy as np
import os
from datetime import datetime, timedelta # obspy 대신 표준 라이브러리 사용

# --- 1. 파일 불러오기 ---
input_filename = "earthquake_features_2024-12-01.csv"
try:
    df = pd.read_csv(input_filename)
    print(f"--- '{input_filename}' 파일을 성공적으로 불러왔습니다. (총 {len(df)}개 이벤트) ---")
except FileNotFoundError:
    print(f"--- 오류: '{input_filename}' 파일을 찾을 수 없습니다. ---")
    exit() # 스크립트 종료

# --- 2. USGS API 요청을 위한 설정 ---
usgs_api_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
search_radius_km = 100  # IRIS와 USGS 간의 위치 오차를 고려한 검색 반경
time_window_seconds = 60 # IRIS와 USGS 간의 시간 오차를 고려한 검색 시간 (±60초)

results_tsunami_flags = [] # USGS 플래그를 저장할 리스트

print(f"--- USGS API 조회를 시작합니다 (총 {len(df)}개 이벤트). 이벤트당 약 1초가 소요됩니다. ---")

# --- 3. DataFrame의 '모든 행'을 순회 ---
for index, row in df.iterrows():

    # CSV에서 현재 이벤트 정보 추출
    event_lat = row['latitude']
    event_lon = row['longitude']
    event_mag = row['magnitude']
    event_time_str = row['time_utc'] # 문자열로 시간 가져오기

    try:
        # CSV의 시간 형식: '2024-01-01T22:27:01.373000'
        event_time_dt = datetime.strptime(event_time_str, '%Y-%m-%dT%H:%M:%S.%f')
    except ValueError:
        # 가끔 마이크로초(.f)가 없는 시간 형식을 대비
        try:
            event_time_dt = datetime.strptime(event_time_str, '%Y-%m-%dT%H:%M:%S')
        except Exception as e:
            print(f"  > 시간 파싱 오류 (행 {index}): {e}. 이 행은 건너뜁니다.")
            results_tsunami_flags.append(np.nan)
            continue

    print(f"\n--- [ {index + 1} / {len(df)} ] 처리 중 (IRIS Mag: {event_mag}) ---")
    print(f"  > IRIS 시간: {event_time_dt.isoformat()}")

    # USGS 검색을 위한 '시공간 창' 설정 (timedelta 사용)
    search_start_dt = event_time_dt - timedelta(seconds=time_window_seconds)
    search_end_dt = event_time_dt + timedelta(seconds=time_window_seconds)

    # API가 요구하는 ISO 형식으로 변환 (UTC 표기 'Z' 추가)
    search_start_iso = search_start_dt.isoformat() + "Z"
    search_end_iso = search_end_dt.isoformat() + "Z"

    # API 요청 파라미터 (!!! minmagnitude 필터 제거 !!!)
    params = {
        'format': 'geojson',
        'starttime': search_start_iso,
        'endtime': search_end_iso,
        'latitude': event_lat,
        'longitude': event_lon,
        'maxradiuskm': search_radius_km
    }

    try:
        # --- 4. USGS API 호출 ---
        response = requests.get(usgs_api_url, params=params, timeout=10)

        usgs_tsunami_flag = np.nan # 기본값은 '데이터 없음'

        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])

            # --- 5. API 응답 결과 처리 ---
            if len(features) == 0:
                # Case 1: 좁은 범위에서 매칭되는 이벤트 '없음'
                print("  > USGS 매칭 실패 (0개). 쓰나미 플래그 '0'으로 설정.")
                usgs_tsunami_flag = 0 # 매칭이 안되면 쓰나미가 아니라고 가정

            elif len(features) == 1:
                # Case 2: 1개로 '정확히' 매칭됨 (Best case)
                props = features[0]['properties']
                usgs_tsunami_flag = props['tsunami']
                print(f"  > USGS 매칭 성공 (1개)! (ID: {features[0]['id']}, Mag: {props['mag']})")
                print(f"  > *** 공식 쓰나미 플래그: {usgs_tsunami_flag} ***")

            else:
                # Case 3: 2개 이상 '모호하게' 매칭됨
                print(f"  > USGS 매칭 모호함 ({len(features)}개 발견). 규모가 가장 비슷한 이벤트로 매칭 시도...")

                best_match = None
                min_mag_diff = float('inf')

                # IRIS의 규모(event_mag)와 가장 차이가 적게 나는 이벤트를 찾는다
                for f in features:
                    usgs_mag = f['properties']['mag']
                    if usgs_mag is None: continue

                    mag_diff = abs(usgs_mag - event_mag)

                    if mag_diff < min_mag_diff:
                        min_mag_diff = mag_diff
                        best_match = f

                if best_match:
                    props = best_match['properties']
                    usgs_tsunami_flag = props['tsunami']
                    print(f"  > > 최적 매칭: (ID: {best_match['id']}, Mag: {props['mag']})")
                    print(f"  > *** 공식 쓰나미 플래그: {usgs_tsunami_flag} ***")
                else:
                    print("  > > 최적 매칭 실패. 플래그 '0'으로 설정.")
                    usgs_tsunami_flag = 0 # 그래도 못찾으면 0

        else:
            # API 자체가 실패한 경우 (e.g., 500 서버 오류)
            print(f"  > USGS API 요청 실패 (HTTP Status: {response.status_code}).")
            usgs_tsunami_flag = np.nan # '0'이 아니라 '데이터 없음'으로 표기

    except requests.RequestException as e:
        # 네트워크 연결 오류 등
        print(f"  > API 요청 중 예외 발생: {e}")
        usgs_tsunami_flag = np.nan # '데이터 없음'

    # 최종 플래그를 리스트에 추가
    results_tsunami_flags.append(usgs_tsunami_flag)

    # !!! USGS API 서버 과부하를 막기 위해 1초 대기 (매우 중요) !!!
    time.sleep(1)

# --- 6. 원본 DataFrame에 새로운 컬럼 추가 ---
df['tsunami_flag_usgs'] = results_tsunami_flags

# --- 7. 새 파일로 저장 ---
output_filename = f"earthquake_features_2024-12-01_Tsunami.csv"
output_path = os.path.join(os.getcwd(), output_filename)
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n--- 작업 완료 ---")
print(f"  > '{output_path}' 파일에 USGS 쓰나미 플래그가 추가된 데이터를 저장했습니다.")
print("\n[최종 데이터 샘플 (tsunami_flag_usgs 컬럼 확인)]")
print(df.head())

# 쓰나미 플래그가 1인 (즉, 쓰나미가 발생한) 이벤트가 있었는지 확인
tsunami_events = df[df['tsunami_flag_usgs'] == 1]
if not tsunami_events.empty:
    print("\n[!!! 쓰나미(1)로 확인된 이벤트 !!!]")
    print(tsunami_events[['time_utc', 'magnitude', 'tsunami_flag_usgs']])
else:
    print("\n[확인: 이 데이터셋에서 쓰나미(1)로 확인된 이벤트는 없습니다.]")

--- 'earthquake_features_2024-12-01.csv' 파일을 성공적으로 불러왔습니다. (총 803개 이벤트) ---
--- USGS API 조회를 시작합니다 (총 803개 이벤트). 이벤트당 약 1초가 소요됩니다. ---

--- [ 1 / 803 ] 처리 중 (IRIS Mag: 4.3) ---
  > IRIS 시간: 2024-12-31T23:28:55.746000
  > USGS 매칭 성공 (1개)! (ID: us6000pj8c, Mag: 4.3)
  > *** 공식 쓰나미 플래그: 0 ***

--- [ 2 / 803 ] 처리 중 (IRIS Mag: 5.0) ---
  > IRIS 시간: 2024-12-31T23:13:20.048000
  > USGS 매칭 성공 (1개)! (ID: us6000pgri, Mag: 5)
  > *** 공식 쓰나미 플래그: 0 ***


In [4]:
import pandas as pd
from geopy.distance import great_circle
import glob
import os
from tqdm import tqdm

# --- 1. 설정 ---
RADIUS_KM = 60.0

# --- 2. 12개 파일 찾기 ---
file_pattern = "earthquake_features_*_Tsunami.csv"
csv_files = glob.glob(file_pattern)

if not csv_files:
    print(f"오류: '{file_pattern}' 패턴의 CSV 파일을 찾을 수 없습니다.")
    exit()

print(f"총 {len(csv_files)}개의 '..._Tsunami.csv' 파일을 찾았습니다. '여진 제거'를 시작합니다...")

# --- 3. 메인 '여진 제거(Declustering)' 루프 ---

# 3-1. 12개 파일 순회
for file_path in tqdm(csv_files, desc="전체 파일 진행률"):

    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"'{file_path}' 파일 읽기 오류: {e}")
        continue

    # (중요) 날짜/시간(time_utc) 열을 datetime 객체로 변환
    try:
        # ★★★ (수정됨) ★★★
        # format='mixed' 대신 errors='coerce' 사용.
        # 변환이 불가능한 '깨진' 날짜는 NaT(Not a Time)로 바꿉니다.
        df['time_utc'] = pd.to_datetime(df['time_utc'], errors='coerce')

        # (추가) NaT로 바뀐, 즉 '깨진' 행은 모두 삭제
        original_rows_before_coerce = len(df)
        df = df.dropna(subset=['time_utc'])
        cleaned_rows_after_coerce = len(df)

        if original_rows_before_coerce != cleaned_rows_after_coerce:
            print(f"\n[{os.path.basename(file_path)}] '깨진' 날짜 데이터 {original_rows_before_coerce - cleaned_rows_after_coerce}행을 제거했습니다.")
        # ★★★★★★★★★★★★★★★★★

    except KeyError:
        print(f"\n오류: '{file_path}'에 'time_utc' 열이 없습니다. 건너뜁니다.")
        continue

    df['day'] = df['time_utc'].dt.day

    indices_to_keep = set()
    processed_indices = set()
    original_row_count = len(df) # 깨끗한 행 기준

    # 3-2. 날짜(day)별로 순회
    for day in df['day'].unique():

        day_df_indices = df[df['day'] == day].index

        # 3-3. 날짜별 $O(n^2)$ 거리 비교
        for i in day_df_indices:

            if i in processed_indices:
                continue

            current_quake = df.loc[i]
            current_coords = (current_quake['latitude'], current_quake['longitude'])

            cluster_indices = [i]

            for j in day_df_indices:
                if i == j:
                    continue

                compare_quake = df.loc[j]
                compare_coords = (compare_quake['latitude'], compare_quake['longitude'])

                dist = great_circle(current_coords, compare_coords).km

                if dist <= RADIUS_KM:
                    cluster_indices.append(j)

            # 3-4. 클러스터 처리
            processed_indices.update(cluster_indices)
            mainshock_index = df.loc[cluster_indices]['magnitude'].idxmax()
            indices_to_keep.add(mainshock_index)

    # 3-5. (덮어쓰기) 살아남은 본진들만 필터링
    df_cleaned = df.loc[list(indices_to_keep)]
    df_cleaned = df_cleaned.drop(columns=['day'])

    # 3-6. (덮어쓰기) 원본 파일 경로에 그대로 저장
    try:
        df_cleaned.to_csv(file_path, index=False)
        print(f"\n[성공] '{os.path.basename(file_path)}' 덮어쓰기 완료.")
        print(f"  -> 원본(정리 후): {original_row_count} 행 => '본진'만: {len(df_cleaned)} 행")
    except Exception as e:
        print(f"\n[실패] '{os.path.basename(file_path)}' 저장 실패: {e}")

print("\n" + "="*50)
print("🎉 모든 파일의 '여진 제거' 및 '덮어쓰기'가 완료되었습니다.")

총 12개의 '..._Tsunami.csv' 파일을 찾았습니다. '여진 제거'를 시작합니다...


전체 파일 진행률:   8%|▊         | 1/12 [00:00<00:05,  2.16it/s]


[성공] 'earthquake_features_2024-01-01_Tsunami.csv' 덮어쓰기 완료.
  -> 원본(정리 후): 533 행 => '본진'만: 532 행


전체 파일 진행률:  17%|█▋        | 2/12 [00:01<00:05,  1.93it/s]


[성공] 'earthquake_features_2024-02-01_Tsunami.csv' 덮어쓰기 완료.
  -> 원본(정리 후): 592 행 => '본진'만: 591 행


...


[성공] 'earthquake_features_2024-10-01_Tsunami.csv' 덮어쓰기 완료.
  -> 원본(정리 후): 521 행 => '본진'만: 521 행


전체 파일 진행률:  92%|█████████▏| 11/12 [00:04<00:00,  2.80it/s]


[성공] 'earthquake_features_2024-11-01_Tsunami.csv' 덮어쓰기 완료.
  -> 원본(정리 후): 482 행 => '본진'만: 482 행


전체 파일 진행률: 100%|██████████| 12/12 [00:04<00:00,  2.58it/s]


[성공] 'earthquake_features_2024-12-01_Tsunami.csv' 덮어쓰기 완료.
  -> 원본(정리 후): 555 행 => '본진'만: 553 행

🎉 모든 파일의 '여진 제거' 및 '덮어쓰기'가 완료되었습니다.
